# Visualización del Dataset Trusted 12x6

Este notebook muestra cómo se estructura y visualiza el dataset trusted que contiene secuencias de 12 frames de entrada y 6 frames de salida.

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys
sys.path.append('..')
from src.visualization.visualizer import setup_custom_cmap

# Configurar matplotlib
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
# Cargar dataset trusted
dataset_path = "../inputs/data_trusted_12x6.h5"
with h5py.File(dataset_path, "r") as h5_file:
    # Mostrar información del dataset
    print("Información del dataset:")
    print(f"Fecha de creación: {h5_file.attrs['created_date']}")
    print(f"Frames de entrada: {h5_file.attrs['input_frames']}")
    print(f"Frames de salida: {h5_file.attrs['output_frames']}")
    print("\nEstructura del archivo:")
    print("- inputs/")
    print("- targets/")
    print("- timestamps/")
    
    # Mostrar número de secuencias
    n_sequences = len(h5_file['inputs'])
    print(f"\nNúmero total de secuencias: {n_sequences}")

In [ ]:
def visualize_sequence(sequence_idx):
    """Visualiza una secuencia completa del dataset"""
    with h5py.File(dataset_path, "r") as h5_file:
        # Obtener datos
        seq_name = f"sequence_{sequence_idx:05d}"
        inputs = h5_file['inputs'][seq_name][:]
        targets = h5_file['targets'][seq_name][:]
        
        # Obtener timestamps
        input_timestamps = [ts.decode() for ts in h5_file['timestamps'][f"{seq_name}_input"][:]]
        target_timestamps = [ts.decode() for ts in h5_file['timestamps'][f"{seq_name}_target"][:]]
        
        # Configurar visualización
        n_rows = 2
        n_cols = max(len(inputs), len(targets))
        fig = plt.figure(figsize=(20, 8))
        
        # Configurar colormap
        custom_cmap = setup_custom_cmap()
        
        # Mostrar frames de entrada
        for i, (frame, ts) in enumerate(zip(inputs, input_timestamps)):
            ax = plt.subplot(n_rows, n_cols, i + 1)
            im = ax.imshow(frame, cmap=custom_cmap, vmin=0, vmax=100)
            ax.set_title(f'Input {i+1}\n{datetime.strptime(ts, "%Y_%m_%d_%H_%M_%S").strftime("%Y-%m-%d %H:%M")}')
            ax.axis('off')
        
        # Mostrar frames objetivo
        for i, (frame, ts) in enumerate(zip(targets, target_timestamps)):
            ax = plt.subplot(n_rows, n_cols, n_cols + i + 1)
            im = ax.imshow(frame, cmap=custom_cmap, vmin=0, vmax=100)
            ax.set_title(f'Target {i+1}\n{datetime.strptime(ts, "%Y_%m_%d_%H_%M_%S").strftime("%Y-%m-%d %H:%M")}')
            ax.axis('off')
        
        # Añadir colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        cbar = plt.colorbar(im, cax=cbar_ax, orientation='vertical')
        cbar.set_label('Intensidad (0-100)', fontsize=10)
        
        plt.suptitle(f'Secuencia {sequence_idx}', y=1.05, fontsize=14)
        plt.tight_layout()
        plt.show()

In [ ]:
# Visualizar algunas secuencias de ejemplo
for idx in range(3):
    visualize_sequence(idx)
    print("\n" + "="*80 + "\n")

## Análisis de las Secuencias

Podemos ver que:
1. Las secuencias de entrada (12 frames) se muestran en la fila superior
2. Las secuencias objetivo (6 frames) se muestran en la fila inferior
3. Cada frame mantiene su timestamp original
4. La intensidad está normalizada entre 0 y 100
5. Se usa la misma paleta de colores que en el entrenamiento